In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
import scipy.io as sio
import numpy as np
import pickle as pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from scipy import signal
import scipy.io
import os
import math

import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
# from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, Conv2D, MaxPooling2D, RepeatVector
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_path = '/content/drive/My Drive/Major/BED_dataset_RAW_PARSED/'

In [ ]:
def load_eeg_data(directory):
    data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith('.mat'):
            filepath = os.path.join(directory, filename)

            mat_contents = scipy.io.loadmat(filepath)
            recordings = mat_contents['recording']

            recordings = recordings[:30208, 2:-1]
            recordings = recordings.T

            data.append(recordings)

            # Extract label from filename
            start_index = filename.index('s') + 1
            end_index = filename.index('_')
            # Extract the substring between 's' and '_'
            number = filename[start_index:end_index]
            person_id = int(number) # Convert the extracted number to an integer

            label = person_id - 1
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

In [ ]:
eeg_data, labels = load_eeg_data(dataset_path)

print("EEG data shape:", eeg_data.shape)
print("Labels shape:", labels.shape)

EEG data shape: (63, 14, 30208)
Labels shape: (63,)


In [ ]:
eeg_Label = []
for i in labels:
    for j in range (0,14):
        eeg_Label.append(i)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(eeg_Label)

y.shape

(882, 21)

In [ ]:
import functools
def combine_dims(a, i=0, n=1):
  s = list(a.shape)
  combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
  return np.reshape(a, s[:i] + [combined] + s[i+n+1:])

In [ ]:
eeg_Data = combine_dims(np.array(eeg_data), 0)

eeg_Data.shape

(882, 30208)

In [ ]:
fs = 256

size_dataset = len(eeg_Data)
nperseg = 512
noverlap = 256
f_size = math.ceil(((nperseg + 1)/2))
t_size = int((eeg_Data[0].size - noverlap ) / (nperseg - noverlap))
X_full = np.ndarray(shape=(size_dataset, f_size, t_size, 3))

for i in range(0, size_dataset):
  for j in range(0,3):
    X = eeg_Data[i]
    f, t, Sxx = spectrogram(X, fs, nperseg = nperseg, noverlap = noverlap, mode='psd')
    X_full[i, :, :, j] = Sxx

In [ ]:

X_full.shape

(882, 257, 117, 3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

X_train /= 255
X_test /= 255

In [ ]:
num_classes = 21

input_shape = (X_train.shape[1],X_train.shape[2],X_train.shape[3])
input_shape

(257, 117, 3)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights based on imbalance ratio
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)

# Convert class_weights array to a dictionary
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, RepeatVector, LSTM, Dense, Layer
import keras.backend as K

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[0][-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[0][1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        lstm_output, _, _ = x
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = lstm_output * at
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][-1])

# Define the model with the corrected Attention layer
inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(inputs)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
x = RepeatVector(4)(x)
lstm_output = LSTM(256, return_sequences=True, return_state=True, dropout=0.2)(x)
attention_output = Attention()(lstm_output)
dense_output = Dense(128, activation='relu')(attention_output)
outputs = Dense(num_classes, activation='softmax')(dense_output)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 257, 117, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 255, 115, 32)         896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 127, 57, 32)          0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 125, 55, 64)          18496     ['max_pooling2d[0][0]']   

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer ="Adam", loss =keras.losses.categorical_crossentropy, metrics=['accuracy'])
m=model.fit(X_train, y_train,epochs=100,batch_size=128,verbose=1,validation_data=(X_test, y_test), class_weight=class_weights_dict, callbacks =[es, mc])

Epoch 1/100
6/6 [==============================] - ETA: 0s - loss: 2.5155 - accuracy: 0.3376 
Epoch 1: val_accuracy improved from -inf to 0.55367, saving model to best_model.h5
6/6 [==============================] - 141s 23s/step - loss: 2.5155 - accuracy: 0.3376 - val_loss: 1.8256 - val_accuracy: 0.5537
Epoch 2/100
6/6 [==============================] - ETA: 0s - loss: 1.3860 - accuracy: 0.6979 
Epoch 2: val_accuracy improved from 0.55367 to 0.72316, saving model to best_model.h5
6/6 [==============================] - 143s 24s/step - loss: 1.3860 - accuracy: 0.6979 - val_loss: 1.2318 - val_accuracy: 0.7232
Epoch 3/100
6/6 [==============================] - ETA: 0s - loss: 0.7637 - accuracy: 0.8397 
Epoch 3: val_accuracy improved from 0.72316 to 0.81356, saving model to best_model.h5
6/6 [==============================] - 148s 25s/step - loss: 0.7637 - accuracy: 0.8397 - val_loss: 0.8138 - val_accuracy: 0.8136
Epoch 4/100
6/6 [==============================] - ETA: 0s - loss: 0.4408 - 

In [ ]:
saved_model = load_model('best_model.h5')

In [ ]:
_, train_f1 = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_f1 = saved_model.evaluate(X_test, y_test, verbose=0)

print(test_f1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(m.history['accuracy'])
plt.plot(m.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
y_test1=np.argmax(y_test, axis=1)

y_pred_saved = saved_model.predict(X_test)
y_pred_saved=np.argmax(y_pred_saved,axis=1)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test1,y_pred_saved))

In [ ]:
saved_model = load_model('best_model.h5')
saved_model.save('/content/drive/My Drive/saved_models/BED_14.h5')